In [1]:
#obtaining top leafsubhaloid

import h5py
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import illustris_python as il
from tqdm import tqdm

def hdf5_to_dict(file_path):
    data_dict = {}
    
    with h5py.File(file_path, 'r') as hdf_file:
        # Recursively traverse the HDF5 file and populate the dictionary
        def traverse(name, obj):
            if isinstance(obj, h5py.Dataset):
                data_dict[name] = obj[()]  # Store dataset value in dictionary
            elif isinstance(obj, h5py.Group):
                # Create a nested dictionary for groups
                group_dict = {}
                data_dict[name] = group_dict
                
                # Recursively visit the group's children
                obj.visititems(traverse)
        
        hdf_file.visititems(traverse)
    
    return data_dict

# ZONA DE TESTEO RANDOM

In [2]:
#datos estelares de todas las particulas
fields = ['Coordinates','ParticleIDs']
basePath = '/virgotng/universe/IllustrisTNG/L35n2160TNG/output'
star_data = il.snapshot.loadSubset(basePath, 99, 'star', fields=fields)

#del star_data['count']

In [7]:
subhaloid_list = [547844, 455291, 478216, 501208, 531910, 570319, 571633, 582137, 589496, 598112, 606820, 619381, 628949]
subhaloid = subhaloid_list[0]

filename = 'tables/subhalo'
filename += str(subhaloid)
filename += '.0.csv'

subhalo_df = pd.read_csv(filename)
subhalo_df.head()

filename = 'tables/subhalo'
filename += str(subhaloid)
filename += '.0_counterrotating.csv'

subhalo_df_counterrotating = pd.read_csv(filename)
subhalo_df_counterrotating.head()

,Unnamed: 0,M,x,y,z,Vx,Vy,Vz,Metallicity,U,StellarFormationTime,R,Jx,Jy,Jz,Total Energy,Circularity,ParticleIDs
0,0,58448.898438,-0.154764,0.074867,0.017551,4.889226,89.534176,-21.100849,2.087895,-247481.671875,0.259017,0.172815,-3.151162,-3.179844,-14.222720,-243238.912385,-0.731022,117199215714
1,1,62360.707031,-0.020649,0.320695,-0.029299,49.370230,11.525117,21.150869,1.606135,-243849.562500,0.262900,0.322692,7.120658,-1.009767,-16.070773,-242340.758919,-0.763500,117529495303
2,2,58892.878906,-0.134683,0.284063,-0.026598,56.775556,62.465438,-1.170615,2.457734,-245603.109375,0.339009,0.315498,1.328940,-1.667789,-24.540860,-242039.726892,-1.165902,122958304653
3,3,41889.089844,0.231612,-0.297637,-0.077891,-44.759405,-12.339432,24.386536,1.216150,-243326.984375,0.377832,0.385096,-8.219477,-2.161859,-16.180036,-241951.799836,-0.768691,127174665332
4,4,45795.941406,-0.385167,0.143833,-0.002027,8.152116,40.101793,-19.987695,2.832675,-242528.828125,0.300566,0.411152,-2.793607,-7.715132,-16.618447,-241491.768760,-0.790647,119574225956


In [4]:
#datos del header de la snapshot, para transformar coords y esas cosas
snap_header = pd.read_csv('snap_header.csv')

snapNum = 99
redshift_z = np.round(snap_header['redshift'][snapNum-1],3)
scalefactor_a = snap_header['scalefactor'][snapNum-1]

#esto no es necesario, ya que h y los omegas son valores fijos en verdad, (._.)
h = snap_header['h'][snapNum-1]
H_0 = h/10
omega_0 = snap_header['omega_0'][snaobtain_subhalo_pospNum-1]
omega_lambda = snap_header['omega_lambda'][snapNum-1]

w_a = omega_0/(omega_lambda*(scalefactor_a**3))
H_a = H_0 * np.sqrt(omega_lambda) * np.sqrt(1 + w_a)
snap_header.head()

,snapnum,redshift,scalefactor,h,omega_0,omega_lambda
0,1.0,14.989173,0.062542,0.6774,0.3089,0.6911
1,2.0,11.980213,0.077040,0.6774,0.3089,0.6911
2,3.0,10.975643,0.083503,0.6774,0.3089,0.6911
3,4.0,9.996591,0.090937,0.6774,0.3089,0.6911
4,5.0,9.388771,0.096258,0.6774,0.3089,0.6911


In [5]:
h,H_0,omega_0,omega_lambda,w_a,H_a

(0.6773999929428101,
 0.06773999929428101,
 0.308899998664856,
 0.691100001335144,
 0.44696859798594785,
 0.06773999929428101)

In [6]:
redshift_z, scalefactor_a

(0.0, 1.0)

In [10]:
#Datos base obtenidos arriba
#subhalo_df dataframe keys ParticleIDs
#stars_data dict keys Coordinates ParticleIDs
#

filtro_contrarotante = np.isin(star_data['ParticleIDs'],subhalo_df_counterrotating['ParticleIDs'])
filtro_contrarotante

array([False, False, False, ..., False, False, False])

In [13]:
star_data['ParticleIDs'][filtro_contrarotante]

array([117199215714, 117529495303, 127174665332, ..., 111278921093,
       101676107873, 101676110689], dtype=uint64)

In [18]:
star_data['Coordinates'][filtro_contrarotante,0],star_data['Coordinates'][filtro_contrarotante,1],star_data['Coordinates'][filtro_contrarotante,2]

(array([7943.74689251, 7943.72458284, 7943.910004  , ..., 8008.83240232,
        7923.07775614, 7933.12140438]),
 array([14833.81912448, 14833.64991372, 14834.02802296, ...,
        14774.48847702, 14874.77593083, 14873.98004177]),
 array([3681.99132753, 3681.9026651 , 3681.73420489, ..., 3645.11578791,
        3703.71173287, 3693.44908228]))

### ZONA DE EJECUCION

In [2]:
#IMPORTANDO FUNCIONES PROPIAS
from particle_follower import load_counterrotating_particles
from particle_follower import load_stellar_particles
from particle_follower import filter_particles_byID
from particle_follower import generate_time_table
from particle_follower import generate_time_tables
from particle_follower import obtain_subhalo_pos

In [3]:
subhaloid_list = [ 455291,  478216,  501208,  531910,  547844,  570319, 571633,  582137,  589496,  598112,  606820,  619381,  628949]
subhaloid = subhaloid_list[12]
generate_time_tables(subhaloid,15,29)

Cargando particulas del subhalo
Cargando particulas en snapshot 15
Comenzando filtrado
cambio del marco de referencia
Tabla para particulas CR 628949 en la snapshot 15 CONSTRUIDA
Cargando particulas en snapshot 16
Comenzando filtrado
cambio del marco de referencia
Tabla para particulas CR 628949 en la snapshot 16 CONSTRUIDA
Cargando particulas en snapshot 17
Comenzando filtrado
cambio del marco de referencia
Tabla para particulas CR 628949 en la snapshot 17 CONSTRUIDA
Cargando particulas en snapshot 18
Comenzando filtrado
cambio del marco de referencia
Tabla para particulas CR 628949 en la snapshot 18 CONSTRUIDA
Cargando particulas en snapshot 19
Comenzando filtrado
cambio del marco de referencia
Tabla para particulas CR 628949 en la snapshot 19 CONSTRUIDA
Cargando particulas en snapshot 20
Comenzando filtrado
cambio del marco de referencia
Tabla para particulas CR 628949 en la snapshot 20 CONSTRUIDA
Cargando particulas en snapshot 21
Comenzando filtrado
cambio del marco de referencia


'Tablas generadas'